# Building the pipeline for working with EEG data

In this notebook
- we build the codes to extract initial raw data;
- we calculate different metrics as mesuares of functional connectivity;
- based on calculated measures like Mutual Information and Coherence we create Adjacency Matrices; 
- we create graphs based on the previously calculated metrics;
- make some plots to visualize the obtained graphs.

MAYBE based on graphs we will train some graph convolutional neural networks...

# 0 - Required packages



# 1 - Datasets Links

- Links for downloading the datasets: https://isip.piconepress.com/projects/tuh_eeg/html/downloads.shtml
- Command for getting the data from the server: 
- "rsync -auxvL nedc@www.isip.piconepress.com:data/eeg/tuh_eeg_abnormal/v2.0.0/ ." 
- "rsync -auxvL nedc@www.isip.piconepress.com:data/eeg/tuh_eeg_epilepsy/v1.0.0/ ."

### Data engineering

In [20]:
from glob2 import glob
import numpy as np
import mne

In [21]:
#edf_file_list = glob("../tuh_eeg_abnormal/edf/*/*/*/*/*/*/*.edf")
edf_file_list = glob("../tuh_eeg_epilepsy/edf/*/*/*/*/*/*.edf")

len(edf_file_list)

1648

In [22]:
file_path = edf_file_list[0]
raw_data = mne.io.read_raw_edf(file_path, verbose=False, preload=False)

In [23]:
edf_file_list[0].split("/")

['..',
 'tuh_eeg_epilepsy',
 'edf',
 'epilepsy',
 '03_tcp_ar_a',
 '076',
 '00007656',
 's010_2011_03_13',
 '00007656_s010_t000.edf']

In [24]:
# extract subject IDs from the file path, create python set to extract unique elements from list, convert to list again 
unique_epilepsy_patient_ids = list(set([x.split("/")[-1].split("_")[0] for x in edf_file_list]))
len(unique_epilepsy_patient_ids)

200

In [25]:
unique_epilepsy_patient_ids[0]

'00010715'